In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import xml.etree.ElementTree as ET
from nltk import word_tokenize
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import pymorphy2
%matplotlib inline

C:\Anaconda3\lib\site-packages\gensim-1.0.1-py3.5-win-amd64.egg\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-06-18 23:58:05,107 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
def tree_df(tree_name):
    tree = ET.parse(tree_name)
    sentences = []
    label = []
    data_item = []
    data = []
    polarity=[]
    root = tree.getroot()
    for sentence in root.iter('sentence'):
        for text in sentence.iter('text'):
            for opinion in sentence.iter('Opinion'):
                data_item.append(text.text)
                data_item.append(opinion.get('category'))
                data_item.append(opinion.get('polarity'))
                data.append(data_item)
                data_item=[]
    df = pd.DataFrame(data, columns = ['sentence', 'category', 'polarity'])
    return df

In [3]:
name = 'se16_ru_rest_train.xml'
df = tree_df(name)
test_name = 'se16_ru_rest_test.xml'
df_test = tree_df(test_name)
morph = pymorphy2.MorphAnalyzer()

2017-06-18 23:58:13,253 : INFO : Loading dictionaries from C:\Anaconda3\lib\site-packages\pymorphy2_dicts\data
2017-06-18 23:58:14,181 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [4]:
from stop_words import get_stop_words
stop_words = get_stop_words('russian')

In [5]:
def classify_SVC(X_train, y_train, X_test, y_test):
    clf = SVC(kernel='linear', random_state=241, probability=True)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    df_pred = pd.Series(y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    return df_pred, accuracy, precision, recall, f1


In [6]:
def classify_logreg(X_train, y_train, X_test, y_test):
    clf = linear_model.LogisticRegression(n_jobs=1, C=1e5)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    df_pred = pd.Series(y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    return df_pred, accuracy, precision, recall, f1


In [7]:
def classify_NB(X_train, y_train, X_test, y_test):
    clf = MultinomialNB()
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    df_pred = pd.Series(y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    return df_pred, accuracy, precision, recall, f1

In [8]:
def classify_random_forest(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier( n_estimators = 100, random_state=241 )
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    df_pred = pd.Series(y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    return df_pred, accuracy, precision, recall, f1

# BAG OF WORDS model

In [9]:
%%time
count_vectorizer = CountVectorizer(stop_words=stop_words, max_features=1000,analyzer="word")
train_1 = count_vectorizer.fit_transform(df['sentence']).toarray()
test_1 = count_vectorizer.transform(df_test['sentence']).toarray()

Wall time: 169 ms


In [10]:
mlb = mlb = MultiLabelBinarizer()
df_all = pd.concat([df, df_test], axis = 0)
categories = mlb.fit_transform(df_all['category'])
train_2 = categories[:len(df)]
test_2 = categories[len(df):]

In [11]:
y_train = df['polarity']
y_test = df_test['polarity']

In [12]:
X_test_bow = np.concatenate((test_1,test_2), axis=1)
X_train_bow = np.concatenate((train_1,train_2), axis=1)

In [13]:
df_pred_svc, accuracy_bow_SVC, precision_bow_SVC, recall_bow_SVC, f1_bow_SVC = classify_SVC(X_train_bow,y_train,X_test_bow,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_bow_SVC, precision_bow_SVC, recall_bow_SVC, f1_bow_SVC))


accuracy = 0.69923, precision = 0.39138, recall = 0.34242, f1 = 0.35225


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
df_pred_logreg, accuracy_bow_logreg, precision_bow_logreg, recall_bow_logreg, f1_bow_logreg = classify_logreg(X_train_bow,y_train,X_test_bow,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_bow_logreg, precision_bow_logreg, recall_bow_logreg, f1_bow_logreg))


accuracy = 0.61923, precision = 0.32338, recall = 0.32369, f1 = 0.32337


In [15]:
df_pred_NB, accuracy_bow_NB, precision_bow_NB, recall_bow_NB, f1_bow_NB = classify_NB(X_train_bow,y_train,X_test_bow,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_bow_NB, precision_bow_NB, recall_bow_NB, f1_bow_NB))


accuracy = 0.71692, precision = 0.38504, recall = 0.33437, f1 = 0.33945


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
df_pred_random_forest, accuracy_bow_random_forest, precision_bow_random_forest, recall_bow_random_forest, f1_bow_random_forest = classify_random_forest(X_train_bow,y_train,X_test_bow,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_bow_random_forest, precision_bow_random_forest, recall_bow_random_forest, f1_bow_random_forest))


accuracy = 0.68769, precision = 0.36779, recall = 0.32085, f1 = 0.32589


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# TF.IDF

In [17]:
tfidfv = TfidfVectorizer(stop_words=stop_words, max_features=1000,analyzer="word")
train_1_tfidf = tfidfv.fit_transform(df['sentence']).toarray()
test_1_tfidf = tfidfv.transform(df_test['sentence']).toarray()

In [18]:
X_test_tfidf = np.concatenate((test_1_tfidf,test_2), axis=1)
X_train_tfidf = np.concatenate((train_1_tfidf,train_2), axis=1)

In [19]:
df_pred_tfidf_SVC, accuracy_tfidf_SVC, precision_tfidf_SVC, recall_tfidf_SVC, f1_tfidf_SVC = classify_SVC(X_train_tfidf,y_train,X_test_tfidf,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_tfidf_SVC, precision_tfidf_SVC, recall_tfidf_SVC, f1_tfidf_SVC))


accuracy = 0.70769, precision = 0.45891, recall = 0.32242, f1 = 0.32843


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
df_pred_tfidf_logreg, accuracy_tfidf_logreg, precision_tfidf_logreg, recall_tfidf_logreg, f1_tfidf_logreg = classify_logreg(X_train_tfidf,y_train,X_test_tfidf,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_tfidf_logreg, precision_tfidf_logreg, recall_tfidf_logreg, f1_tfidf_logreg))


accuracy = 0.62000, precision = 0.32228, recall = 0.32121, f1 = 0.32141


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
df_pred_tfidf_NB, accuracy_tfidf_NB, precision_tfidf_NB, recall_tfidf_NB, f1_tfidf_NB = classify_NB(X_train_tfidf,y_train,X_test_tfidf,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_tfidf_NB, precision_tfidf_NB, recall_tfidf_NB, f1_tfidf_NB))


accuracy = 0.71462, precision = 0.35275, recall = 0.31266, f1 = 0.30750


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
df_pred_tfidf_forest, accuracy_tfidf_forest, precision_tfidf_forest, recall_forest_NB, f1_tfidf_forest = classify_random_forest(X_train_tfidf,y_train,X_test_tfidf,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_tfidf_forest, precision_tfidf_forest, recall_forest_NB, f1_tfidf_forest))


accuracy = 0.69077, precision = 0.38021, recall = 0.31052, f1 = 0.31294


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Word2Vec

In [23]:
sen_clear = []
for sentence in np.array(df['sentence']):
    sen_cl = re.sub(r'[^\w\d\s\-\n\{\}]', " ", sentence, flags=re.M | re.U | re.I)
    #words = sen_cl.lower().split()
    words = sen_cl.split()
    words_c = []
    for word in words:
        if word != '-':
            words_c.append(word)
    words_stop = []
    for word in words_c:
        for stop_word in stop_words:
            if word != stop_word:
                words_stop.append(word)
                break
    sen_clear.append(words_stop)
sen_clear_test = []
for sentence in np.array(df_test['sentence']):
    sen_cl = re.sub(r'[^\w\d\s\-\n\{\}]', " ", sentence, flags=re.M | re.U | re.I)
    #words = sen_cl.lower().split()
    words = sen_cl.split()
    words_c = []
    for word in words:
        if word != '-':
                words_c.append(word)
    words_stop = []
    for word in words_c:
        for stop_word in stop_words:
            if word != stop_word:
                words_stop.append(word)
                break
    sen_clear_test.append(words_stop)

In [24]:
model = gensim.models.Word2Vec(sen_clear,size=1000,window=5,min_count=5, workers=4)

2017-06-19 00:04:50,357 : INFO : collecting all words and their counts
2017-06-19 00:04:50,371 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-19 00:04:50,403 : INFO : collected 7328 word types from a corpus of 60535 raw words and 4089 sentences
2017-06-19 00:04:50,404 : INFO : Loading a fresh vocabulary
2017-06-19 00:04:50,431 : INFO : min_count=5 retains 2127 unique words (29% of original 7328, drops 5201)
2017-06-19 00:04:50,436 : INFO : min_count=5 leaves 51029 word corpus (84% of original 60535, drops 9506)
2017-06-19 00:04:50,457 : INFO : deleting the raw counts dictionary of 7328 items
2017-06-19 00:04:50,469 : INFO : sample=0.001 downsamples 51 most-common words
2017-06-19 00:04:50,470 : INFO : downsampling leaves estimated 41857 word corpus (82.0% of prior 51029)
2017-06-19 00:04:50,471 : INFO : estimated required memory for 2127 words and 1000 dimensions: 18079500 bytes
2017-06-19 00:04:50,488 : INFO : resetting layer weights
2017-06-19 00:0

In [25]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,))
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords != 0.0:
        featureVec = np.true_divide(featureVec,nwords)
    else:
        featureVec = np.zeros((num_features,))
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features))
    #reviewFeatureVecs = np.array((len(reviews),num_features))
    # 
    # Loop through the reviews
    for i in range(len(reviews)):
        reviewFeatureVecs[i] = makeFeatureVec(reviews[i], model, num_features)
        
    #for review in reviews:

       # Call the function (defined above) that makes average feature vectors
       #reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
       #    num_features)
       #
       # Increment the counter
       #counter = counter + 1.
    
    return reviewFeatureVecs

In [26]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.


trainDataVecs = getAvgFeatureVecs( sen_clear, model, 1000 )

print ("Creating average feature vecs for test reviews")

testDataVecs = getAvgFeatureVecs( sen_clear_test, model, 1000 )

Creating average feature vecs for test reviews


In [27]:
X_test_w2v = np.concatenate((testDataVecs,test_2), axis=1)
X_train_w2v = np.concatenate((trainDataVecs,train_2), axis=1)

In [28]:
df_pred_w2v_SVC, accuracy_w2v_SVC, precision_w2v_SVC, recall_w2v_SVC, f1_w2v_SVC = classify_SVC(X_train_w2v,y_train,X_test_w2v,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_SVC, precision_w2v_SVC, recall_w2v_SVC, f1_w2v_SVC))


df_pred_w2v_logreg, accuracy_w2v_logreg, precision_w2v_logreg, recall_w2v_logreg, f1_w2v_logreg = classify_logreg(X_train_w2v,y_train,X_test_w2v,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_logreg, precision_w2v_logreg, recall_w2v_logreg, f1_w2v_logreg))


#df_pred_w2v_NB, accuracy_w2v_NB, precision_w2v_NB, recall_w2v_NB, f1_w2v_NB = classify_NB(X_train_w2v,y_train,X_test_w2v,y_test)
#print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_NB, precision_w2v_NB, recall_w2v_NB, f1_w2v_NB))


df_pred_w2v_forest, accuracy_w2v_forest, precision_w2v_forest, recall_w2v_forest, f1_w2v_forest = classify_random_forest(X_train_w2v,y_train,X_test_w2v,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_forest, precision_w2v_forest, recall_w2v_forest, f1_w2v_forest))


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy = 0.66923, precision = 0.16731, recall = 0.25000, f1 = 0.20046
accuracy = 0.72538, precision = 0.44596, recall = 0.33756, f1 = 0.34715
accuracy = 0.68385, precision = 0.31461, recall = 0.27610, f1 = 0.25522


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
df_pred_w2v_forest, accuracy_w2v_forest, precision_w2v_forest, recall_w2v_forest, f1_w2v_forest = classify_random_forest(X_train_w2v,y_train,X_test_w2v,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_forest, precision_w2v_forest, recall_w2v_forest, f1_w2v_forest))


accuracy = 0.68385, precision = 0.31461, recall = 0.27610, f1 = 0.25522


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [30]:
from sklearn.naive_bayes import GaussianNB
def classify_NB(X_train, y_train, X_test, y_test):
    clf = GaussianNB()
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    df_pred = pd.Series(y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'micro')
    recall = metrics.recall_score(y_test, y_pred, average = 'micro')
    f1=metrics.f1_score(y_test, y_pred, average = 'micro')
    return df_pred, accuracy, precision, recall, f1

In [85]:
df_pred_w2v_NB, accuracy_w2v_NB, precision_w2v_NB, recall_w2v_NB, f1_w2v_NB = classify_NB(X_train_w2v,y_train,X_test_w2v,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_NB, precision_w2v_NB, recall_w2v_NB, f1_w2v_NB))

accuracy = 0.41308, precision = 0.25676, recall = 0.26899, f1 = 0.22037


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
